In [ ]:
import numpy as np
import os
import pandas as pd
import grid2op
from grid2op.Chronics import ChangeNothing
from chronix2grid.kpi.Generator_parameter_checker import EnergyMix_AprioriChecker
import plotly
import plotly.graph_objects as go
from grid2op.PlotGrid import NUKE_COLOR, THERMAL_COLOR, WIND_COLOR, SOLAR_COLOR, HYDRO_COLOR
import json
import copy

# for pandas interactive plots
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)


In [ ]:
env_name = "l2rpn_icaps_2021_small"
env = grid2op.make(env_name)

gen_p = []
for months in ["Scenario_april_000", "Scenario_august_001", "Scenario_december_000", 
               "Scenario_february_000", "Scenario_january_000", "Scenario_july_000",
               "Scenario_june_001", "Scenario_march_000", "Scenario_may_000",
               "Scenario_november_000", "Scenario_october_000", "Scenario_september_000"]:
    env.set_id(months)
    env.reset()
    gen_p.append( 1.0 * env.chronics_handler.real_data.data.prod_p)
gen_p = np.concatenate(gen_p)

In [ ]:
gen_solar_id = np.where(env.gen_type == "solar")[0]
gen_wind_id = np.where(env.gen_type == "wind")[0]
gen_hydro_id = np.where(env.gen_type == "hydro")[0]
gen_nuclear_id = np.where(env.gen_type == "nuclear")[0]
gen_thermal_id = np.where(env.gen_type == "thermal")[0]
proportion_solar = pd.DataFrame({"total_solar": gen_p[:,gen_solar_id].sum(axis=1),
                                      "total_wind": gen_p[:,gen_wind_id].sum(axis=1),
                                      "total_hydro": gen_p[:,gen_hydro_id].sum(axis=1),
                                      "total_nuclear": gen_p[:,gen_nuclear_id].sum(axis=1),
                                      "total_thermal": gen_p[:,gen_thermal_id].sum(axis=1),
                                     })

In [ ]:
labels2 = ['solar','wind','hydro', "nuclear", "thermal"]
values2 = [proportion_solar["total_solar"].sum(),
           proportion_solar["total_wind"].sum(),
           proportion_solar["total_hydro"].sum(),
           proportion_solar["total_nuclear"].sum(),
           proportion_solar["total_thermal"].sum(),
          ]

fig = go.Figure(data=[go.Pie(labels=labels2,
                             values=values2,
                             marker_colors=[SOLAR_COLOR, WIND_COLOR, HYDRO_COLOR, NUKE_COLOR, THERMAL_COLOR],
                            text=[f"{round(el / 12., -3):,.0f} MWh" for el in values2]
                            )]
                             
                )
fig.update_layout(
    title=f"Share of energy produced depending on energy type for environment {env_name}"
)
fig.show()